<a href="https://colab.research.google.com/github/gusadi09/K-Nearest-Neighbour/blob/main/tupro3AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import numpy as np
from collections import Counter
import math

In [ ]:
df = pd.read_csv('Diabetes.csv')

In [ ]:
def scaling(data) :
    scale = MinMaxScaler(feature_range=(0, 1))
    
    data_scale = scale.fit_transform(data)
    
    normalisasi = pd.DataFrame(data_scale)
    normalisasi = normalisasi.rename(columns={0:'Pregnancies',1:'Glucose', 2:'BloodPressure',3:'SkinThickness', 4:'Insulin',5:'BMI', 6:'DiabetesPedigreeFunction',7:'Age', 8:'Outcome'})
  
    return normalisasi

In [ ]:
#Data Train dan Data Test

def preProccesSplit(dataFrame, range_train1, range_train2, range_test1, range_test2) :
  dt = scaling(dataFrame)
  train1 = dt.iloc[range_train1:range_train2]
  test1 = dt.iloc[range_test1:range_test2]

  return train1, test1

def preProcesSplitConcat(dataFrame, range_train1_1, range_train1_2, range_train2_1, range_train2_2, range_test1, range_test2) :
  dt = scaling(dataFrame)
  train1_1 = dt.iloc[range_train1_1:range_train1_2]
  train1_2 = dt.iloc[range_train2_1:range_train2_2]
  train1 = pd.concat([train1_1, train1_2])
  test1 = dt.iloc[range_test1:range_test2]

  return train1, test1

In [ ]:
def euclideanDst(dt_train, dt_tes) :
  n1 = dt_train[0] - dt_tes[0]
  n2 = dt_train[1] - dt_tes[1]
  n3 = dt_train[2] - dt_tes[2]
  n4 = dt_train[3] - dt_tes[3]
  n5 = dt_train[4] - dt_tes[4]

  a = np.hypot(n1, n2)
  b = np.hypot(n3, n4)
  c = np.hypot(a, b)

  return np.hypot(c, n5)

In [ ]:
def neighborDst(dt_train, dt_test) :
  tVal = []
  temp = []

  for i in range(0, len(dt_test)) :
    for j in range(0, len(dt_train)) :
      temp.append([euclideanDst(dt_train[j], dt_test[i]), dt_train[j][8]])
    tVal.append(temp)
    temp = []

  return tVal

In [ ]:
def knn(neighbor, k) :
  temp = []
  kn = []

  def indexSort(dt) :
    return dt[0]
  
  for i in range(len(neighbor)) :
    neighbor[i].sort(key = indexSort)

  for i in range(len(neighbor)) :
    for j in range(0, k) :
      temp.append(neighbor[i][j])
    kn.append(temp)
    temp = []

  return kn

In [ ]:
def prediksi(test, train, k) :
  dst = neighborDst(train, test)
  nearNeighbor = knn(dst, k)

  nearNeighborVal = []
  tmp = []

  for i in range(len(nearNeighbor)) :
    for j in range(len(nearNeighbor[i])) :
      tmp.append(nearNeighbor[i][j][1])
    nearNeighborVal.append(tmp)
    tmp = []

  result = []

  for i in range(len(nearNeighborVal)) :
    data = Counter(nearNeighborVal[i])
    result.append(data.most_common(1)[0][0])
  
  return result

In [ ]:
def hitAkurasi(dt_tes, prediksi) :
  tst_set = dt_tes['Outcome'].values.tolist()
  correct = 0

  for i in range(len(prediksi)) :
    if tst_set[i] == prediksi[i] :
      correct += 1
  
  hasil = (correct/float(len(tst_set))) * 100.0

  return hasil

In [ ]:
def foldVal(dt_tes, dt_train) :
  x = dt_train.to_numpy()
  y = dt_tes.to_numpy()

  k = []

  for i in range(len(y)) :
    k.append(hitAkurasi(dt_tes, prediksi(y, x, i+1)))

  return [k.index(max(k)), max(k)]

In [ ]:
def rataRata(val1, val2, val3, val4, val5) :
  rata_rata = (CrossVal1[1] + CrossVal2[1] + CrossVal3[1] + CrossVal4[1] + CrossVal5[1]) / 5
  return rata_rata

Main Program

In [ ]:
#data test dan train 1
train1, tst1 = preProccesSplit(df, 0, 613, 614, 767)
CrossVal1 = foldVal(tst1, train1)
print("Data Test 1 dan Data Train 1")
print("K : ",CrossVal1[0],", Accuracy : ", CrossVal1[1])
print("========================================================================")
print()

#data test dan train 2
train2, tst2 = preProcesSplitConcat(df, 0, 460, 614, 767, 461, 640)
CrossVal2 = foldVal(tst2, train2)
print("Data Test 2 dan Data Train 2")
print("K : ",CrossVal2[0],", Accuracy : ", CrossVal2[1])
print("========================================================================")
print()

#data test dan train 3
train3, tst3 = preProcesSplitConcat(df, 0, 306, 461, 767, 307, 460)
CrossVal3 = foldVal(tst3, train3)
print("Data Test 3 dan Data Train 3")
print("K : ",CrossVal3[0],", Accuracy : ", CrossVal3[1])
print("========================================================================")
print()

#data test dan train 4
train4, tst4 = preProcesSplitConcat(df, 0, 153, 307, 767, 154, 306)
CrossVal4 = foldVal(tst4, train4)
print("Data Test 4 dan Data Train 4")
print("K : ",CrossVal4[0],", Accuracy : ", CrossVal4[1])
print("========================================================================")
print()

#data test dan train 5
train5, tst5 = preProccesSplit(df, 154, 767, 0, 153)
CrossVal5 = foldVal(tst5, train5)
print("Data Test 5 dan Data Train 5")
print("K : ",CrossVal5[0],", Accuracy : ", CrossVal5[1])
print("========================================================================")
print()

r = rataRata(CrossVal1[0], CrossVal2[0], CrossVal3[0], CrossVal4[0], CrossVal5[0])
print("Rata-rata akurasi kNN : ", r)

Data Test 1 dan Data Train 1
K :  27 , Accuracy :  75.81699346405229

Data Test 2 dan Data Train 2
K :  18 , Accuracy :  83.79888268156425

Data Test 3 dan Data Train 3
K :  59 , Accuracy :  75.81699346405229

Data Test 4 dan Data Train 4
K :  2 , Accuracy :  68.42105263157895

Data Test 5 dan Data Train 5
K :  7 , Accuracy :  78.43137254901961

Rata-rata akurasi kNN :  76.45705895805348
